In [1]:
#Mount GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel as parallel
import torch.optim as optim
import torch.utils.data 
from torch.autograd import Variable

##Importing Dataset

In [3]:
movies = pd.read_csv('drive/MyDrive/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding= 'latin-1')

In [4]:
users = pd.read_csv('drive/MyDrive/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding= 'latin-1')

In [5]:
ratings = pd.read_csv('drive/MyDrive/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding= 'latin-1')

##Preparing the Training and Test Set

In [6]:
training_set = pd.read_csv('drive/MyDrive/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('drive/MyDrive/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

###Getting the number of users and movies

In [7]:
 total_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
 total_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
 print("total users: ", total_users)
 print("total movies: ", total_movies)

total users:  943
total movies:  1682


###Converting the data into an array with users in lines and movies in columns

In [8]:
def convert(dataset):
  user_movies = []
  for user_id in range(1,total_users + 1):
    movie_ids = dataset[:,1][dataset[:,0] == user_id]
    rating_ids = dataset[:,2][dataset[:,0] == user_id]
    rating_list = np.zeros(total_movies)
    rating_list[movie_ids - 1] = rating_ids
    user_movies.append(list(rating_list))
  return user_movies

In [9]:
training_set = convert(training_set)
test_set = convert(test_set)

###Convert the data into Torch tensors

In [10]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

###Convert the ratings into binary ratings

In [11]:
training_set[training_set == 0] = -1 #ratings 0 are invalid
#rating 1 or 2 are not liked, hence 0
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set > 2] = 1

In [12]:
test_set[test_set == 0] = -1 #ratings 0 are invalid
#rating 1 or 2 are not liked, hence 0
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set > 2] = 1

###Creating the architecture of Neural Network

In [73]:
class RBM:
  def __init__(self, no_visibleNodes, no_hiddenNodes):
    self.weights = torch.randn(no_hiddenNodes, no_visibleNodes)
    self.bias_h = torch.randn(1, no_hiddenNodes)
    self.bias_v = torch.randn(1, no_visibleNodes)
  
  def sample_hidden(self, x):
    wx = torch.mm(x, self.weights.t())
    activation = wx + self.bias_h.expand_as(wx)
    probability_activated_hidden_given_visible = torch.sigmoid(activation)
    return probability_activated_hidden_given_visible, torch.bernoulli(probability_activated_hidden_given_visible)
  
  def sample_visible(self, y):
    wy = torch.mm(y, self.weights)
    activation = wy + self.bias_v.expand_as(wy)
    probability_activated_visible_given_hidden = torch.sigmoid(activation)
    return probability_activated_visible_given_hidden, torch.bernoulli(probability_activated_visible_given_hidden)
  
  def train(self, visible_0, visible_k, probability_hidden_0, probability_hidden_k):
    v0 = torch.mm(visible_0.t(), probability_hidden_0)
    vk = torch.mm(visible_k.t(), probability_hidden_k)
    # self.weights += (v0-vk)
    self.bias_v += torch.sum((visible_0-visible_k), 0)
    self.bias_h += torch.sum((probability_hidden_0 - probability_hidden_k), 0)

In [74]:
no_visibleNodes = len(training_set[0])
no_hiddenNodes = 100
batch_size = 100

In [75]:
rbm = RBM(no_visibleNodes, no_hiddenNodes)

###Training the RBM

In [76]:
epochs = 10 
for e in range(1, epochs+1):
  train_loss = 0 
  counter = 0.
  
  for user_id in range(0, total_users - batch_size, batch_size):
    visible_k = training_set[user_id: user_id + batch_size]
    visible_0 = training_set[user_id: user_id + batch_size]
    probability_hidden_0,_ = rbm.sample_hidden(visible_0) #here '_' is used to make all other returned elements as None except probability_hidden_0
    
    for k in range(10):
      _,probability_hidden_k = rbm.sample_hidden(visible_k)
      _,visible_k = rbm.sample_visible(probability_hidden_k)
      visible_k[visible_0 < 0] = visible_0[visible_0 < 0]
    
    probability_hidden_k,_ = rbm.sample_hidden(visible_k)
    
    
    rbm.train(visible_0, visible_k, probability_hidden_0, probability_hidden_k)
    train_loss += torch.mean(torch.abs(visible_k[visible_0 >= 0] - visible_0[visible_0 >= 0]))
    counter += 1.
  
  print('epoch: ' + str(e) + ' loss: ' + str(train_loss/counter))

epoch: 1 loss: tensor(0.2951)
epoch: 2 loss: tensor(0.2511)
epoch: 3 loss: tensor(0.2525)
epoch: 4 loss: tensor(0.2484)
epoch: 5 loss: tensor(0.2492)
epoch: 6 loss: tensor(0.2473)
epoch: 7 loss: tensor(0.2478)
epoch: 8 loss: tensor(0.2496)
epoch: 9 loss: tensor(0.2476)
epoch: 10 loss: tensor(0.2507)


###Testing the RBM

In [78]:
test_loss = 0 
counter = 0.

for user_id in range(total_users):
  visible = training_set[user_id: user_id + 1]
  visible_t = test_set[user_id: user_id + 1]
  
  if len(visible_t[visible_t>=0]) > 0:
    _,hidden = rbm.sample_hidden(visible)
    _,visible = rbm.sample_visible(hidden)
  
    test_loss += torch.mean(torch.abs(visible_t[visible_t >= 0] - visible[visible_t >= 0]))
    counter += 1.

print('test loss: ' + str(test_loss/counter))

test loss: tensor(0.2505)
